In [1]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [2]:
chain = "ethereum"
attacks = [#("0x14d8Ada7A0BA91f59Dc0Cb97C8F44F1d177c2195","ethereum","tominu","2023-01-18","2023-01-27"),
           #("0x69f0EdC352eCffC4EF49516c9b20eA88B3E947cb,0xceed34f03a3e607cc04c2d0441c7386b190d7cf4","ethereum","blueclues","2023-01-19","2023-01-26"),
           #("0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527","ethereum","quaternion","2023-01-17","2023-01-19"),
           #("0x8a2d94ea342cbdd6d57db614b24f20cae286cac6,0x22898dc59145eae79471dd1f06b7e542e0982d30","ethereum","Upswing","2022-12-21","2023-01-20"),
           #("0x15d87DC2Eb27fdA26451f8FB04C576639104344d","ethereum","roefinance","2023-01-10","2023-01-18"),
           #("0x9D0163e76BbCf776001E639d65F573949a53AB03","ethereum","lendhub","2023-01-10","2023-01-16"),
           #("0xc578d755cd56255d3ff6e92e1b6371ba945e3984","binance","ufdaoattack","2023-01-01","2023-02-06")
           ("0x7021c1b142eb634fa0749cda270c7aff74dc3b7f","ethereum","DEPUSDT","2023-06-06","2023-06-15"),
           ]

# chain = "binance"
# attacks = [#("0x66be80c796cba0844dace3e291632bfd397bd7a0","binance","phyproxy","2023-01-01","2023-01-24"),
#            #("0x1ae2dc57399b2f4597366c5bf4fe39859c006f99","binance","thoreum","2023-01-14","2023-01-20"),
#            #("0x9BbD94506398a1459F0Cd3B2638512627390255e,0xda5919bf3a49ad47b7c7103a9ed3902cee78d528","binance","omniprotocol","2023-01-07","2023-01-18"),
#            ("0xc578d755cd56255d3ff6e92e1b6371ba945e3984","binance","ufdaoattack","2023-01-01","2023-02-06"),
#            ("0x67a909f2953fb1138beA4B60894B51291D2d0795,0xE2Ba15be8C6Fb0d7C1F7bEA9106eb8232248FB8B","binance","bratoken","2022-12-08","2023-01-16"),
#            #("0xcf2362b46669e04b16d0780cf9b6e61c82de36a7","binance","GDS","2023-01-01","2023-01-05")
#             ]

In [3]:
# #clustering
# for attacker_address in attacker_addresses.split(","):
#     sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
#     print(sql)
#     df = execute_query(sql)
#     print(df)

In [4]:
def get_alert_hashes(attacker_addresses: str, start_date: str, end_date: str) -> pd.DataFrame:

    alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
    for attacker_address in attacker_addresses.split(","):
        url = 'https://api.forta.network/graphql'
        query = '''query exampleQuery {
            # first 5 alerts
            alerts(
                input: { first: 2000 , addresses: ["'''+attacker_address+'''"],
                blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
                }
            ) {
                pageInfo {
                hasNextPage
                endCursor {
                    alertId
                    blockNumber
                }
                }
                alerts {
                hash
                createdAt
                
                }
            }
            }
        '''
        r = requests.post(url, json={'query': query})
        #print(r.text)
        json_data = json.loads(r.text)
        df = pd.DataFrame(json_data['data']['alerts']['alerts'])
        #print(df)
        alert_hashes_df = pd.concat([alert_hashes_df, df])

    
    return alert_hashes_df


In [12]:
!npm


zsh:1: command not found: npm


In [13]:

for attacker_addresses, chain, name, start_date, end_date in attacks:

    alert_hashes_df = get_alert_hashes(attacker_addresses.lower(), start_date, end_date)
    print(f"Got {len(alert_hashes_df)} alerts.")
    !echo 'output.txt' > {name}_.txt

    #run npm run to get and route to output.txt
    count = 0
    alert_hashes = ""
    for alert_hash in alert_hashes_df["hash"]:
        print(f"npm run alert {alert_hash} >> {name}.txt 2>> {name}_{chain}.txt")
        !npm run alert {alert_hash} >> {name}.txt 2>> {name}_{chain}.txt


    !grep 'ERROR'  {name}_{chain}.txt >  {name}_{chain}_results.txt
    !grep 'WARN'  {name}_{chain}.txt >>  {name}_{chain}_results.txt
    !grep 'adding cluster mapping'  {name}_{chain}.txt >>  {name}_{chain}_results.txt
    !grep 'adding FP mitigation cluster'  {name}_{chain}.txt >>  {name}_{chain}_results.txt
    !grep 'anomaly'  {name}_{chain}.txt >>  {name}_{chain}_results.txt
    



41
Got 41 alerts.
npm run alert 0x91cafe6f506c0a43d5c522f6b4d45d95dbc3784c3c000d0eba61a846dc98b158 >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0x465620f14b05398612f7b7f964992f60c751febe6da86714d15f4c9e8bf04a4a >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0xdc501d2cea0c1d41d16515c4b00e17f65c73b1f8810bd037df1b9415b5096b3e >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0x95bc70062ec9be11c3992bfab4f395824d3524620836b2850a8a32dfb3edbdb3 >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0x052a6794de5d827623bcc2882bd2d665283d4300a45870013bdd48356539b5d7 >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0xfd4167073e8eca027c1ffc1eee3f132d35aea071cc54ce118b0616c369ee010c >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0x499527841faf22cfc9c3210bd68d46b0699aa646b5ff69ed93335795c20da2ba >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 0x1813641ef3abc00366abfd96697485c735766fdd86eec6621c257fe47c97e56d >> DEPUSDT.txt 2>> DEPUSDT_ethereum.txt
npm run alert 

In [6]:
!grep 'ERROR' {name}.txt


In [7]:
!grep 'WARN' {name}.txt

In [8]:
!grep 'adding cluster mapping' {name}.txt

In [9]:
!grep 'adding FP mitigation cluster' {name}.txt

In [10]:
#read output.txt and answer questions around
!grep 'anomaly' {name}.txt


#anomaly scores
#FP filtering
